In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install konlpy

In [3]:
# !pip install openai langchain langchain-google-genai

In [4]:
# !pip install openai

In [5]:
import pandas as pd
import re
from konlpy.tag import Okt
from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer, GPT2Model, AdamW, get_linear_schedule_with_warmup
# from langchain_google_genai import ChatGoogleGenerativeAI
import torch
from sklearn.preprocessing import LabelEncoder

In [6]:
df = pd.read_excel('/content/drive/MyDrive/dataset/한국어_연속적_대화_데이터셋.xlsx')
df.head(5)

,시작,대화,감정
0,S,아 진짜! 사무실에서 피지 말라니깐! 간접흡연이 얼마나 안좋은데!,분노
1,NaN,그럼 직접흡연하는 난 얼마나 안좋겠니? 안그래? 보면 꼭... 지 생각만 하고.,혐오
2,NaN,손님 왔어요.,중립
3,NaN,손님? 누구?,중립
4,NaN,몰라요. 팀장님 친구래요.,중립


In [7]:
df['시작'] = df['시작'].replace('S', 1)
df.head(5)

,시작,대화,감정
0,1.0,아 진짜! 사무실에서 피지 말라니깐! 간접흡연이 얼마나 안좋은데!,분노
1,NaN,그럼 직접흡연하는 난 얼마나 안좋겠니? 안그래? 보면 꼭... 지 생각만 하고.,혐오
2,NaN,손님 왔어요.,중립
3,NaN,손님? 누구?,중립
4,NaN,몰라요. 팀장님 친구래요.,중립


In [8]:
df = df.fillna({'시작': 0,'대화':' '})

In [9]:
print(df.isnull().sum())

시작    0
대화    0
감정    7
dtype: int64


In [10]:
# 텍스트 정제
def clean_text(text):
    # HTML 태그 제거
    text = re.sub(r'<.*?>', '', text)
    # 이메일 주소 제거
    text = re.sub(r'\S+@\S+', '', text)
    # URL 제거
    text = re.sub(r'http\S+|www\.\S+', '', text)
    # 특수 문자 및 숫자 제거, 한글과 기본 공백을 제외하고 모두 제거
    text = re.sub(r'[^가-힣\s]', '', text)
    # 공백 정리 (중복 공백 제거)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [11]:
# 불용어 리스트
stop_words = set(['은', '는', '이', '가', '을', '를', '에', '의', '로', '도', '으로', '와', '과', '에서', '하다'])

def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stop_words])

# 불용어 제거 적용
df['대화'] = df['대화'].apply(remove_stopwords)

In [12]:
# 형태소 분석
okt = Okt()

def tokenize(text):
    return ' '.join(okt.morphs(text))

# 형태소 분석 적용
df['대화'] = df['대화'].apply(tokenize)

In [13]:
# 이전 대화 내용과 감정 컬럼 생성
df['이전 대화'] = df['대화'].shift(1)
df['이전 감정'] = df['감정'].shift(1)

# 첫 번째 대화는 이전 대화가 없으므로 NaN으로 표시됩니다.
# 필요에 따라 이러한 행을 제거할 수 있습니다.
df = df.dropna().reset_index(drop=True)

print(df)

        시작                                                 대화  감정  \
0      0.0  그럼 직접 흡연 하 는 난 얼마나 안좋겠니 ? 안 그래 ? 보면 꼭 ... 지 생각...  혐오   
1      0.0                                           손님 왔어요 .  중립   
2      0.0                                          손님 ? 누구 ?  중립   
3      0.0                                몰라요 . 팀 장님 친구 래 요 .  중립   
4      0.0                                     내 친구 ? 친구 누구 ?  중립   
...    ...                                                ...  ..   
55608  1.0                                   얘 긴 다 끝났냐 ? 원 예부  중립   
55609  0.0          예 . 그거 때문 에 , 부탁 이 있 ...... 는 ...... 데요 .  중립   
55610  0.0                             여자 숨겨 달라 는 거 면 사절 이다 .  중립   
55611  0.0                                        아무래도 안되나요 ?  중립   
55612  0.0  그 여자 랑 내 가 무슨 상관 인데 ? 아까는 탐정 님 이 부탁 하기에 너 구 하는...  중립   

                                                   이전 대화 이전 감정  
0             아 진짜 ! 사무실 에서 피지 말 라니깐 ! 간접흡연 이 얼마나 안좋은데 !    분노  
1      그럼 직접 흡연 하 는 난 얼마나 안좋겠니 ? 안 그래 ? 보

In [14]:
def emotion_change(prev_emotion, current_emotion):
    positive_emotions = ['행복', '놀람']
    negative_emotions = ['슬픔', '공포', '혐오', '분노']

    # 변화 없음
    if prev_emotion == current_emotion:
        return '변화 없음'
    # 긍정적 변화
    elif prev_emotion == '중립' and current_emotion in positive_emotions:
        return '긍정적 변화'
    # 부정적 변화
    elif prev_emotion == '중립' and current_emotion in negative_emotions:
        return '부정적 변화'
    # 감정 안정
    elif prev_emotion in negative_emotions and current_emotion == '중립':
        return '감정 안정'
    # 감정 강화
    elif prev_emotion in negative_emotions and current_emotion in negative_emotions:
        return '감정 강화'
    else:
        return '기타'

In [15]:
# 예시 데이터프레임에 '감정 변화' 컬럼 추가
df['감정 변화'] = df.apply(lambda x: emotion_change(x['이전 감정'], x['감정']), axis=1)

In [16]:
df.head(5)

,시작,대화,감정,이전 대화,이전 감정,감정 변화
0,0.0,그럼 직접 흡연 하 는 난 얼마나 안좋겠니 ? 안 그래 ? 보면 꼭 ... 지 생각...,혐오,아 진짜 ! 사무실 에서 피지 말 라니깐 ! 간접흡연 이 얼마나 안좋은데 !,분노,감정 강화
1,0.0,손님 왔어요 .,중립,그럼 직접 흡연 하 는 난 얼마나 안좋겠니 ? 안 그래 ? 보면 꼭 ... 지 생각...,혐오,감정 안정
2,0.0,손님 ? 누구 ?,중립,손님 왔어요 .,중립,변화 없음
3,0.0,몰라요 . 팀 장님 친구 래 요 .,중립,손님 ? 누구 ?,중립,변화 없음
4,0.0,내 친구 ? 친구 누구 ?,중립,몰라요 . 팀 장님 친구 래 요 .,중립,변화 없음


In [17]:
from sklearn.model_selection import train_test_split

X = df[['이전 대화', '대화']]  # 입력 데이터
y = df['감정 변화']  # 예측 대상

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
# 모델과 토크나이저 초기화

# tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# model = GPT2Model.from_pretrained('gpt2')
from transformers import GPT2ForSequenceClassification, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=7)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
# 대화 쌍을 입력으로, 감정 변화를 라벨로 사용
# 토크나이저에 패딩 토큰 설정
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer(list(X_train['이전 대화'] + " [SEP] " + X_train['대화']),
                   return_tensors='pt',
                   padding=True,
                   truncation=True,
                   max_length=512)

# 라벨 인코더 초기화
# label_encoder = LabelEncoder()
# emotions = ['행복', '슬픔', '중립', '공포', '혐오', '분노', '놀람']
# # 라벨 인코딩
# encoded_labels = label_encoder.fit_transform(emotions)
# # 인코딩 결과 출력
# print(list(zip(emotions, encoded_labels)))

In [20]:
print(tokenizer.pad_token_id)

50256


In [21]:
from sklearn.preprocessing import LabelEncoder
import torch

# 라벨 인코더 초기화
label_encoder = LabelEncoder()

# y_train에 라벨 인코딩 적용(감정라벨)
y_train_encoded = label_encoder.fit_transform(y_train)

# 인코딩된 라벨을 확인하기 위해 인코딩된 라벨과 원래 라벨을 출력
for original, encoded in zip(y_train[:10], y_train_encoded[:10]):
    print(f"Original: {original}, Encoded: {encoded}")

# 인코딩된 라벨을 PyTorch 텐서로 변환
labels = torch.tensor(y_train_encoded.tolist())

Original: 감정 강화, Encoded: 0
Original: 감정 안정, Encoded: 1
Original: 변화 없음, Encoded: 4
Original: 감정 강화, Encoded: 0
Original: 감정 강화, Encoded: 0
Original: 변화 없음, Encoded: 4
Original: 변화 없음, Encoded: 4
Original: 변화 없음, Encoded: 4
Original: 긍정적 변화, Encoded: 2
Original: 변화 없음, Encoded: 4


In [22]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset

# 데이터셋 생성
dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'], labels)
train_dataloader = DataLoader(dataset, sampler=RandomSampler(dataset), batch_size=4)

# 옵티마이저 설정
optimizer = AdamW(model.parameters(), lr=5e-5)

# 훈련 에폭 설정
epochs = 4

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [25]:
import os
import torch

def save_checkpoint(epoch, model, optimizer, file_path):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }
    torch.save(checkpoint, file_path)

def load_checkpoint(file_path, model, optimizer):
    checkpoint = torch.load(file_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    return checkpoint['epoch']

# 체크포인트 경로
checkpoint_path = 'path/to/checkpoint.pt'

# 체크포인트 파일이 존재하면 로드
if os.path.isfile(checkpoint_path):
    start_epoch = load_checkpoint(checkpoint_path, model, optimizer) + 1


In [26]:
from transformers import AdamW, get_linear_schedule_with_warmup
import torch
from torch.utils.data import DataLoader, RandomSampler, TensorDataset
import numpy as np

# 데이터셋 준비
dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'], labels)
train_dataloader = DataLoader(dataset, sampler=RandomSampler(dataset), batch_size=8)

# 옵티마이저 설정
optimizer = AdamW(model.parameters(), lr=2e-5)

# 총 훈련 스텝 계산
total_steps = len(train_dataloader) * epochs

# 스케줄러 설정
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=total_steps)

# GPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

start_epoch = 0
checkpoint_path = 'path/to/checkpoint.pt'

# 체크포인트 파일이 존재하면 로드
if os.path.isfile(checkpoint_path):
    start_epoch = load_checkpoint(checkpoint_path, model, optimizer) + 1

# 훈련 루프
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    print('-' * 10)

    total_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_attention_mask, b_labels = batch

        model.zero_grad()

        outputs = model(b_input_ids,
                        attention_mask=b_attention_mask,
                        labels=b_labels)

        loss = outputs[0]
        total_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Average training loss: {avg_train_loss:.2f}")
    save_checkpoint(epoch, model, optimizer, checkpoint_path)

    # 검증 루프 추가 가능


Epoch 1/4
----------


AssertionError: Cannot handle batch sizes > 1 if no padding token is defined.